Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Question Answering on the SQuAD Dataset using BERT


# Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Deep Learning Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

The table below provides some reference running time on different machine configurations.  

|QUICK_RUN|Machine Configurations|Running time|
|:---------|:----------------------|:------------|
|True|4 **CPU**s, 14GB memory| ~ 10 minutes |
|True|1 NVIDIA Tesla K80 GPUs, 12GB GPU memory| ~ 3 minutes |
|False|4 NVIDIA Tesla K80 GPUs, 48GB GPU memory| ~ 18 hours |
|False|4 NVIDIA Tesla V100 GPUs, 64GB GPU memory| ~ 7 hours|

If you run into CUDA out-of-memory error, try reducing the `BATCH_SIZE` and `MAX_SEQ_LENGTH`, but note that model performance will be compromised. 

In [3]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = False

## Summary
This notebook demonstrates how to fine tune [pretrained BERT model](https://github.com/huggingface/pytorch-transformers) for extractive question answering task. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation. 

BERT[\[1\]](#References) is a powerful pre-trained lanaguage model that can be used for multiple NLP tasks, including text classification, question answering, named entity recognition, etc. It's able to achieve state of the art performance with only a few epochs of fine tuning on task specific datasets.  
The figure below illustrates how BERT can be fine tuned for extractive question answering task. The question and paragraph tokens are concatenated as a single input token sequence with a special token [SEP] between them. For the paragraph tokens, BERT predicts the probabilities of each token being the start and end of the answer span. The tokens with the highest sum of starting probability and ending probability define the span of the predicted answer

<img src="https://nlpbp.blob.core.windows.net/images/bert_qa.PNG">

In [4]:
import os
import sys

import torch
import numpy as np

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.squad import load_pandas_df
from utils_nlp.models.transformers.question_answering import AnswerExtractor
from utils_nlp.models.transformers.qa_utils import (QADataset, 
                                                    get_qa_dataloader, 
                                                    postprocess_answer, 
                                                    evaluate_qa, 
                                                    TOKENIZER_CLASSES
                                                   )
from utils_nlp.common.timer import Timer

## Configurations

In [5]:
TRAIN_DATA_USED_PERCENT = 1
DEV_DATA_USED_PERCENT = 1
NUM_EPOCHS = 2

if QUICK_RUN:
    TRAIN_DATA_USED_PERCENT = 0.001
    DEV_DATA_USED_PERCENT = 0.01
    NUM_EPOCHS = 1

if torch.cuda.is_available() and torch.cuda.device_count() >= 4:
    MAX_SEQ_LENGTH = 384
    DOC_STRIDE = 128
    BATCH_SIZE = 8
else:
    MAX_SEQ_LENGTH = 128
    DOC_STRIDE = 64
    BATCH_SIZE = 2

print("Max sequence length: {}".format(MAX_SEQ_LENGTH))
print("Document stride: {}".format(DOC_STRIDE))
print("Batch size: {}".format(BATCH_SIZE))
    
SQUAD_VERSION = "v1.1" 
CACHE_DIR = "./temp"

# MODEL_NAME = "bert-large-uncased-whole-word-masking"
# DO_LOWER_CASE = True

MODEL_NAME = "xlnet-large-cased"
DO_LOWER_CASE = False

MAX_QUESTION_LENGTH = 64
LEARNING_RATE = 3e-5

DOC_TEXT_COL = "doc_text"
QUESTION_TEXT_COL = "question_text"
ANSWER_START_COL = "answer_start"
ANSWER_TEXT_COL = "answer_text"
QA_ID_COL = "qa_id"
IS_IMPOSSIBLE_COL = "is_impossible"

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if  torch.cuda.device_count() > 0:
    torch.cuda.manual_seed_all(RANDOM_SEED)

Max sequence length: 384
Document stride: 128
Batch size: 8


## Load Data

### The SQuAD Dataset
Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable. [\[2, 3\]](#References)

<img src="https://nlpbp.blob.core.windows.net/images/squad.png">

There has been two versions of SQuAD datasets. SQuAD 1.1 contains 100,000+ question-answer pairs on 500+ articles. SQuAD 2.0 adds 50,000 new, unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. These datasets are available at [https://rajpurkar.github.io/SQuAD-explorer/](https://rajpurkar.github.io/SQuAD-explorer/). Each dataset comes with a training dataset and a development dataset. 


The utility function `load_pandas_df` downloads the dataset specified by `squad_version` and `file_split` to `local_cache_path` if it doesn't exist already.

In [6]:
train_df = load_pandas_df(local_cache_path=".", squad_version="v1.1", file_split="train")
dev_df = load_pandas_df(local_cache_path=".", squad_version="v1.1", file_split="dev")

In [5]:
train_df.head()

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,5733be284776f41900661182,False
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,5733be284776f4190066117f,False
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,5733be284776f41900661180,False
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,5733be284776f41900661181,False
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,5733be284776f4190066117e,False


In [6]:
dev_df.head()

,doc_text,question_text,answer_start,answer_text,qa_id,is_impossible
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"[177, 177, 177]","[Denver Broncos, Denver Broncos, Denver Broncos]",56be4db0acb8001400a502ec,False
1,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"[249, 249, 249]","[Carolina Panthers, Carolina Panthers, Carolin...",56be4db0acb8001400a502ed,False
2,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"[403, 355, 355]","[Santa Clara, California, Levi's Stadium, Levi...",56be4db0acb8001400a502ee,False
3,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"[177, 177, 177]","[Denver Broncos, Denver Broncos, Denver Broncos]",56be4db0acb8001400a502ef,False
4,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"[488, 488, 521]","[gold, gold, gold]",56be4db0acb8001400a502f0,False


In [7]:
train_df = train_df.sample(frac=TRAIN_DATA_USED_PERCENT).reset_index(drop=True)
dev_df = dev_df.sample(frac=DEV_DATA_USED_PERCENT).reset_index(drop=True)

In [7]:
# train_dataset = QADataset(df=train_df,
#                           doc_text_col=DOC_TEXT_COL,
#                           question_text_col=QUESTION_TEXT_COL,
#                           qa_id_col=QA_ID_COL,
#                           is_impossible_col=IS_IMPOSSIBLE_COL,
#                           answer_start_col=ANSWER_START_COL,
#                           answer_text_col=ANSWER_TEXT_COL)
dev_dataset = QADataset(df=dev_df,
                        doc_text_col=DOC_TEXT_COL,
                        question_text_col=QUESTION_TEXT_COL,
                        qa_id_col=QA_ID_COL,
                        is_impossible_col=IS_IMPOSSIBLE_COL,
                        answer_start_col=ANSWER_START_COL,
                        answer_text_col=ANSWER_TEXT_COL)

## Tokenize and Preprocess Data

The `tokenizer_qa` method of `Tokenizer` tokenizes the input paragraph, question, and answer texts and converts them into the format required by pre-trained BERT model, involving the following steps:
* WordPiece tokenization.
* Convert character-based answer span indices to token-based indices.
* Truncate the question token list if it's longer than `max_question_length`.
* Split the paragraph into multiple segments if it's longer than `max_len` - `max_question_length` - 3. (The "-3" is for the special [CLS] token and two [SEP] tokens.)
* Add the special tokens [CLS] and [SEP].
* Pad the concatenated token sequence to `max_len` if it's shorter.
* Convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary.

In additional to the features required by BERT, `tokenize_qa` outputs a few additional fields needed by postprocessing. See the `QAFeatures` class in [qa_utils.py](../../utils_nlp/models/bert/qa_utils.py) for more details

In [8]:
# train_dataloader = get_qa_dataloader(train_dataset, 
#                                     model_name=MODEL_NAME, 
#                                     is_training=True,
#                                     to_lower=DO_LOWER_CASE,
#                                     batch_size=BATCH_SIZE
#                                         )

dev_dataloader = get_qa_dataloader(dev_dataset, 
                                   model_name=MODEL_NAME, 
                                   is_training=False,
                                   to_lower=DO_LOWER_CASE,
                                   batch_size=BATCH_SIZE)

## Train BERTQAExtractor

In [10]:
qa_extractor = AnswerExtractor(model_name=MODEL_NAME, cache_dir=CACHE_DIR)

100%|██████████| 1441285815/1441285815 [00:26<00:00, 53758449.43B/s]


In [11]:
with Timer() as t:
    qa_extractor.fit(train_dataloader=train_dataloader,
                     num_epochs=NUM_EPOCHS,
                     learning_rate=LEARNING_RATE,
                     cache_model=True)
print("Training time : {:.3f} hrs".format(t.interval / 3600))

# qa_extractor = AnswerExtractor(model_name=MODEL_NAME, cache_dir=CACHE_DIR, load_model_from_dir="./temp")
 

Iteration:   0%|          | 0/10950 [00:00<?, ?it/s]/data/anaconda/envs/nlp_gpu/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

Iteration:  40%|███▉      | 4354/10950 [1:55:30<2:53:19,  1.58s/it]


Iteration:  80%|████████  | 8777/10950 [3:52:01<57:14,  1.58s/it]


Iteration:  22%|██▏       | 2391/10950 [1:02:52<3:44:56,  1.58s/it]


Iteration:  62%|██████▏   | 6813/10950 [2:59:18<1:48:53,  1.58s/it]


Epoch: 100%|██████████| 2/2 [9:37:08<00:00, 17320.54s/it]  58s/it]


Training time : 9.623 hrs


## Predict
Note that the `BERTQAExtractor.predict` only outputs the probabilities of each token being the start and end of the answer span. the `postprocess_answers` method takes these probabilities and generates the final answers. 

In [9]:
qa_extractor = AnswerExtractor(model_name=MODEL_NAME, cache_dir=CACHE_DIR, load_model_from_dir="./temp/distributed_0")
qa_results = qa_extractor.predict(dev_dataloader)

Evaluating: 100%|██████████| 1322/1322 [15:43<00:00,  1.76it/s]


## Postprocess and Generate the Final Answers

In [10]:
# tokenizer_class = TOKENIZER_CLASSES["xlnet"]
# tokenizer = tokenizer_class.from_pretrained(
#     MODEL_NAME, do_lower_case=DO_LOWER_CASE, cache_dir=CACHE_DIR
# )
# final_answers, answer_probs, nbest_answers = postprocess_answer(qa_results,
#                                                                 "./cached_qa_features/cached_examples_test.jsonl",
#                                                                 "./cached_qa_features/cached_features_test.jsonl", 
#                                                                 do_lower_case=DO_LOWER_CASE,
#                                                                 model_type='xlnet',
#                                                                 tokenizer=tokenizer,
#                                                                 n_best_size=5
#                                                                )

In [14]:
for i in [0, 10, 100]:
    print('Paragraph:')
    print(dev_df.iloc[i]['doc_text'])
    print()
    print('Question:')
    print(dev_df.iloc[i]['question_text'])
    print()
    print('Ground truth answers:')
    print(dev_df.iloc[i]['answer_text'])
    print()
    print('Predicted answer:')
    print(final_answers[dev_df.iloc[i]['qa_id']])
    print()
    print('Top N best answers')
    print(nbest_answers[dev_df.iloc[i]['qa_id']])
    print('-------------------------------------------------------------------------------------------------------------------')

Paragraph:
Immunology is strongly experimental in everyday practice but is also characterized by an ongoing theoretical attitude. Many theories have been suggested in immunology from the end of the nineteenth century up to the present time. The end of the 19th century and the beginning of the 20th century saw a battle between "cellular" and "humoral" theories of immunity. According to the cellular theory of immunity, represented in particular by Elie Metchnikoff, it was cells – more precisely, phagocytes – that were responsible for immune responses. In contrast, the humoral theory of immunity, held, among others, by Robert Koch and Emil von Behring, stated that the active immune agents were soluble components (molecules) found in the organism’s “humors” rather than its cells.

Question:
What two scientists were proponents of the humoral theory of immunity?

Ground truth answers:
['Robert Koch and Emil von Behring', 'Robert Koch and Emil von Behring', 'Robert Koch and Emil von Behring,'

## Evaluate

Question answering task is usually evaluated on two metrics: exact match (EM) and F1 score.   
The exact match is computed by first performing some simple normalization (e.g. remove punctuation and convert to lower case) on the ground truth and predicted answers and check if they match exactly after normalization.   
F1 score is computed from token-level precision and recall by comparing the ground truth and predicted answers. 

In [11]:
evaluation_result = evaluate_qa(qa_ids=dev_df['qa_id'], 
                                actuals=dev_df['answer_text'], 
                                preds=final_answers)

{
  "exact": 75.91296121097446,
  "f1": 83.19559419987176,
  "total": 10570,
  "HasAns_exact": 75.91296121097446,
  "HasAns_f1": 83.19559419987176,
  "HasAns_total": 10570
}


In [16]:
# from utils_nlp.models.transformers.qa_utils import QADataset
# from torch.utils.data import (
#     Dataset,
#     IterableDataset,
#     DataLoader,
#     RandomSampler,
#     SequentialSampler,
#     TensorDataset,
# )

# qa_dataset = QADataset(train_df,
#                        doc_text_col="doc_text",
#                        question_text_col="question_text",
#                        qa_id_col="qa_id",
#                        is_impossible_col="is_impossible",
#                        answer_start_col="answer_start",
#                        answer_text_col="answer_text")
# sampler = SequentialSampler(qa_dataset)
# data_loader = DataLoader(qa_dataset, sampler=sampler, batch_size=32)
# def test_generator():
#     features = []
#     c = 0
#     f = True
#     for i in range(10):
#         features.append(c)
#         features.append(c+1)
        
#         while len(features) > 0:
#             output = features[0]
#             features = features[1:]
            
#             if f:
#                 yield output
#             else:
#                 yield output * 10
            
#             f = not f
#         c += 2

# g = test_generator()
# for item in g:
#     print(item)

# from torch.utils.data import TensorDataset
# def test_generator():
#     i = 0
#     while i < 2:
#         i+=1
#         t1 = torch.tensor([list(range(1024)), list(range(1024))], dtype=torch.long)
#         t2 = torch.tensor([list(range(512)), list(range(512))], dtype=torch.long)
#         yield (t1, t2)
        
# g = test_generator()

# for t1, t2 in g:
#     print(t2)

## References

1. Devlin, Jacob and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina, [*BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding*](https://arxiv.org/abs/1810.04805), ACL, 2018.
2. Pranav Rajpurkar, Jian Zhang, Konstantin Lopyrev, Percy Liang, [*SQuAD: 100,000+ Questions for Machine Comprehension of Text*](https://arxiv.org/abs/1606.05250), EMNLP, 2016.
3. Pranav Rajpurkar, Robin Jia, Percy Liang, [*Know What You Don't Know: Unanswerable Questions for SQuAD*](https://arxiv.org/abs/1806.03822), ACL, 2018